In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [50]:
df = pd.read_csv(r"C:\Users\pubal\Downloads\titanic.csv")
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [51]:
#only keeping the columns which we require for naive bayes algorithm
df = df[['Pclass','Sex','Age','Fare','Survived']]
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [52]:
#converting sex from text to boolean format as naive bayes doesnt work on non numeric columns
dummies = pd.get_dummies(df['Sex'])
dummies.head()

,female,male
0,False,True
1,True,False
2,True,False
3,True,False
4,False,True


In [54]:
#concatinating dummies with our original dataframe
df = pd.concat([df,dummies],axis = 'columns')
df.head()

,Pclass,Sex,Age,Fare,Survived,female,male
0,3,male,22.0,7.2500,0,False,True
1,1,female,38.0,71.2833,1,True,False
2,3,female,26.0,7.9250,1,True,False
3,1,female,35.0,53.1000,1,True,False
4,3,male,35.0,8.0500,0,False,True


In [55]:
#finally removing the sex and also 'male' column to avoid rpetitions
df = df.drop(columns = ['Sex','male'])
df.head()

,Pclass,Age,Fare,Survived,female
0,3,22.0,7.2500,0,False
1,1,38.0,71.2833,1,True
2,3,26.0,7.9250,1,True
3,1,35.0,53.1000,1,True
4,3,35.0,8.0500,0,False


In [56]:
#creating inputs and target as X and y for our train test and split
inputs = df.drop(columns = 'Survived')
target = df['Survived']

In [61]:
#wait! there are null values in age column
inputs.head(10)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False
5,3,NaN,8.4583,False
6,1,54.0,51.8625,False
7,3,2.0,21.0750,False
8,3,27.0,11.1333,True
9,2,14.0,30.0708,True


In [60]:
target.head()

0    0
1    1
2    1
3    1
4    0
5    0
6    0
7    0
8    1
9    1
Name: Survived, dtype: int64

In [62]:
#filling input values with means
inputs['Age'] = inputs['Age'].fillna(inputs['Age'].mean(numeric_only = True))
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [80]:
#using train test split to train our model and naive bayes algorithm to predict the outcome
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(inputs,target,test_size = 0.3)
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [81]:
model.fit(X_train,y_train)

GaussianNB()

In [82]:
#we have a fair score on 0.78! Not very good but okayish
model.score(X_test,y_test)

0.7611940298507462

In [83]:
#results of survivors or deaths amongst passengers from titanic passengers dataframe!
model.predict(X_test[0:10])

array([1, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)

In [84]:
#checking cross val score of our model
from sklearn.model_selection import cross_val_score
score = cross_val_score(GaussianNB(),X_train,y_train,cv = 5)
np.average(score)

0.7881935483870969

In [87]:
#understanding why our result is the way it is using probability
model.predict_proba(X_test[0:10])

array([[0.48756542, 0.51243458],
       [0.9072573 , 0.0927427 ],
       [0.79702398, 0.20297602],
       [0.18811788, 0.81188212],
       [0.46761673, 0.53238327],
       [0.4086705 , 0.5913295 ],
       [0.79466842, 0.20533158],
       [0.96396959, 0.03603041],
       [0.97311704, 0.02688296],
       [0.96396959, 0.03603041]])